In [ ]:
import urbanpy as up
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import radians

# Adquisicion de datos

## Limites de la ciudad

In [ ]:
sjl_poly = up.download.download_osm(0, 'San Juan de Lurigancho, Lima')

## Hexagonos de la cuidad

In [ ]:
sjl_hexs, sjl_hexs_centroids = up.geom.gen_hexagons(9, sjl_poly) # approx 0.1053325 km2 hexagons

## Poblacion estimada por hexagono

In [ ]:
pop_peru = up.download.download_hdx("4e74db39-87f1-4383-9255-eaf8ebceb0c9/resource/317f1c39-8417-4bde-a076-99bd37feefce/download/population_per_2018-10-01.csv.zip")
pop_sjl = up.geom.filter_population(pop_peru, sjl_poly)

sjl_hexs_pop = up.geom.merge_shape_hex(sjl_hexs, pop_sjl, how='inner', op='within', agg={'population_2020': 'sum'})
sjl_hexs_pop.plot(column='population_2020', missing_kwds={'color': 'grey'})

In [ ]:
sjl_hexs.loc[sjl_hexs_pop.index, 'population_2020'] = sjl_hexs_pop['population_2020'].fillna(0).values

## Datos geoespaciales de mercados 

In [ ]:
sjl_markets =  gpd.read_file('inputs/flp_sjl/selected_facilities_sjl.shp') # contains markets & possible markets positions
sjl_markets = sjl_markets.to_crs('EPSG:4326')

In [ ]:
# Get old markets positions
sjl_old_markets = sjl_markets[sjl_markets['is_market']==1]
sjl_old_markets.shape

## Datos geoespaciales de posibles mercados (parques y lozas)

In [ ]:
parques_lozas = gpd.read_file('outputs/parques_lozas_lima_metropolitana/parques_lozas_lima_metropolitana/parques_lozas_lima_metropolitana.shp')
parques_lozas.crs = 'EPSG:4326'

In [ ]:
parques_lozas.head()

# Calculo de indicadores

## Movimiento (estimado) de personas por mercado

In [ ]:
from scipy.spatial import cKDTree

In [ ]:
def osrm_routes(origin, destination, profile):
    try:
        orig = f'{origin.x},{origin.y}'
        dest = f"{destination.x},{destination.y}"
        url = f'http://localhost:5000/route/v1/{profile}/{orig};{dest}' # Local osrm server
        response = requests.get(url, params={'overview': 'false'})
        data = response.json()['routes'][0]
        return [data['distance'], data['duration']]
    except Exception as err:
        print(err)
        print(response.reason)
        print(response.url)

In [ ]:
# Creamos un KDTree para buscar el vecino espacial más cercano
kdtree = cKDTree(data=sjl_old_markets[['lon','lat']].values)

In [ ]:
sjl_hexs_centroids.loc[:,'nn_market'] = sjl_hexs_centroids.geometry.apply(
    lambda geom: kdtree.query([geom.x, geom.y])[1]
)

In [ ]:
import requests

In [ ]:
from tqdm import tqdm 

In [ ]:
tqdm.pandas()

In [ ]:
# Distancia (km) y duración (seconds) del viaje a pie
sjl_hexs_centroids[['dist_nn_market_walk', 'dur_nn_market_walk']] = sjl_hexs_centroids.progress_apply(
    lambda row: osrm_routes(
        origin=row.geometry, 
        destination = sjl_old_markets.iloc[row['nn_market']].geometry,
        profile = 'walking'
    ),
    result_type='expand',
    axis=1,
)

In [ ]:
(sjl_hexs_centroids[['dist_nn_market_walk', 'dur_nn_market_walk']] / [1000, 60]).describe()

In [ ]:
# Agregar las columnas al GeoDataFrame hexágonos
sjl_hexs.loc[sjl_hexs_centroids.index,'dist_nn_market_walk'] = sjl_hexs_centroids['dist_nn_market_walk'].values / 1000 # meters to km
sjl_hexs.loc[sjl_hexs_centroids.index,'dur_nn_market_walk'] = sjl_hexs_centroids['dur_nn_market_walk'].values / 60 # seconds to minutes

In [ ]:
sjl_hexs.loc[sjl_hexs_centroids.index,'nn_market'] = sjl_hexs_centroids['nn_market'].values

In [ ]:
sjl_old_markets_proj = sjl_old_markets.to_crs('EPSG:4326')

In [ ]:
sjl_old_markets_proj_in_hex = gpd.sjoin(
    sjl_old_markets_proj,
    sjl_hexs,
    how = 'left',
    op='within'
)

In [ ]:
markets_per_hex = sjl_old_markets_proj_in_hex.groupby('index_right')['id'].count()

In [ ]:
sjl_hexs.loc[markets_per_hex.index, 'n_markets'] = markets_per_hex.values
sjl_hexs['n_markets'] = sjl_hexs['n_markets'].fillna(0)

In [ ]:
import math

def create_duration_labels(durations):
    default_bins = [0, 15, 30, 45, 60, 90, 120]
    default_labels = ["Menos de 15", "De 15 a 30", "De 30 a 45", "De 45 a 60", "De 60 a 90", "De 90 a 120", "Más de 120"]

    bins_ = default_bins.copy()

    max_duration_raw = durations.max()
    max_duration_asint = math.ceil(max_duration_raw)

    bins_.insert(0, max_duration_asint)
    bins_ = sorted(set(bins_))
    ix = bins_.index(max_duration_asint)
    
    if (ix + 1) >= len(default_bins) and max_duration_asint != 120:
        default_bins.append(max_duration_asint)
    
    return default_bins[:ix + 1], default_labels[:ix]

In [ ]:
custom_bins, custom_labels = create_duration_labels(sjl_hexs['dur_nn_market_walk'])

# Generamos cortes en la variable duración del viaje
sjl_hexs['dur_nn_market_walk_bins'] = pd.cut(
    sjl_hexs['dur_nn_market_walk'], 
    bins=custom_bins,
    labels=custom_labels
)

In [ ]:
# Verificamos la cantidad de celdas por corte
sjl_hexs['dur_nn_market_walk_bins'].value_counts()

In [ ]:
# Visualize results
sjl_hexs.plot(
    column='dur_nn_market_walk_bins',
    cmap='magma_r',
    legend=True, 
    figsize=(5,7.5)
)
plt.show()

In [ ]:
sjl_hexs = sjl_hexs.rename(columns={0:'hex'}) # format

In [ ]:
sjl_hexs.drop('dur_nn_market_walk_bins', axis=1).to_file('outputs/sjl_hex.geojson', driver='GeoJSON') # save results

In [ ]:
sjl_hexs = gpd.read_file('outputs/sjl_hex.geojson', driver='GeoJSON')

In [ ]:
from mapboxkey import MAPBOX_API_KEY

In [ ]:
import plotly.express as px
px.set_mapbox_access_token(MAPBOX_API_KEY)
fig = px.choropleth_mapbox(
    sjl_hexs.reset_index(), geojson=sjl_hexs.geometry.__geo_interface__, locations='index', 
    color='population_2020',
    color_continuous_scale='viridis',
    mapbox_style="streets",
    zoom=11, 
    center = {
       "lat": sjl_hexs.geometry.unary_union.centroid.y,
       "lon": sjl_hexs.geometry.unary_union.centroid.x
    },
    opacity=0.3,
    labels={
        'population_2020': 'Población Estimada 2020',
        'n_markets': '# de mercados'
    },
    hover_data = ['n_markets']
)
fig.update_layout(mapbox_bearing=-50, 
                  margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.choropleth_mapbox(
    sjl_hexs.reset_index(), geojson=sjl_hexs.geometry.__geo_interface__, locations='index', 
    color='dur_nn_market_walk',
    color_continuous_scale='magma_r',
    mapbox_style="carto-positron",
    zoom=11, 
    center = {
       "lat": sjl_hexs.geometry.unary_union.centroid.y,
       "lon": sjl_hexs.geometry.unary_union.centroid.x
    },
    opacity=0.3,
    labels={
        'dur_nn_market_walk': 'Duración',
        'dist_nn_market_walk': 'Distancia',
        'n_markets': '# de mercados'
    },
    hover_data = ['dur_nn_market_walk', 'dist_nn_market_walk', 'n_markets']
)
fig.update_layout(mapbox_bearing=-50, 
                  margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## Resultados del modelo FLP

In [ ]:
flp_data = gpd.read_file('/Users/cortega/Documents/COVID-19/streamlit-market-location-covid19/inputs/flp_sjl/selected_facilities_sjl.shp')

In [ ]:
flp_data = flp_data.to_crs('EPSG:4326')

In [ ]:
flp_data.head()

## Filtramos los mercados temporales (parques y lozas)

In [ ]:
active_temporal_markets = flp_data[(flp_data['is_market']==0) & (flp_data['active']==1)]

### Obtenemos el poligono correspondiente para cada observacion

In [ ]:
active_temporal_markets_poly = gpd.sjoin(
    parques_lozas,
    active_temporal_markets,
    how='inner',
    op='intersects'
)

In [ ]:
addresses = gpd.tools.geocoding.reverse_geocode(
    points=gpd.points_from_xy(active_temporal_markets_poly['lon'], active_temporal_markets_poly['lat']),
    provider='Nominatim',
)

In [ ]:
active_temporal_markets_poly['address'] = addresses['address'].values

In [ ]:
active_temporal_markets_poly.head()

## Filtramos los mercados actuales

In [ ]:
import numpy as np

In [ ]:
import plotly.graph_objects as go

## Obtenemos la poblacion en los hexagonos cercanos al mercado

In [ ]:
pop_by_market = sjl_hexs.groupby('nn_market')['population_2020'].sum()

In [ ]:
sjl_old_markets['pop_by_market'] = 0
for ix, value in pop_by_market.iteritems():
    sjl_old_markets['pop_by_market'].iloc[ix] = value

In [ ]:
sjl_old_markets['pop_by_market'].hist()

### Obtenemos el aforo de cada mercado

In [ ]:
market_db = pd.read_csv('/Users/cortega/Documents/COVID-19/urbanpy/notebooks/input/market_db.csv')
market_db = gpd.GeoDataFrame(market_db, geometry=gpd.points_from_xy(market_db['longitude'], market_db['latitude']))
market_db.crs = 'EPSG:4326'

In [ ]:
market_db_sjl = market_db[market_db.within(sjl_poly.geometry[0])]

In [ ]:
market_db_sjl.shape

In [ ]:
sjl_old_markets.shape

In [ ]:
sjl_old_markets.head()

In [ ]:
sjl_old_markets_merged = pd.merge(
    sjl_old_markets,
    market_db_sjl[['NOMBRE_MERCADO', 'latitude','longitude','Tipo de mercado', 'Area construida']],
    how='left',
    left_on = ['lat', 'lon'],
    right_on=['latitude', 'longitude']
)

In [ ]:
sjl_old_markets_merged['aforo'] = sjl_old_markets_merged.apply(
    lambda row: row['Area construida']*2 if row['Tipo de mercado']=='Minorista' else row['Area construida']*5,
    axis=1
)

In [ ]:
sjl_old_markets_merged['aforo'].describe()

In [ ]:
sjl_old_markets_merged['aforo_scaled'] = ((sjl_old_markets_merged['aforo'] - sjl_old_markets_merged['aforo'].min() )
 / (sjl_old_markets_merged['aforo'].max() - sjl_old_markets_merged['aforo'].min())) * 20 + 10

In [ ]:
sjl_old_markets_merged['aforo_scaled'].hist()

In [ ]:
sjl_old_markets_merged = sjl_old_markets_merged.dropna()

In [ ]:
sjl_old_markets_merged.head()

In [ ]:
sjl_old_markets_merged.to_file('sjl_old_markets_merged.geojson', driver='GeoJSON')

In [ ]:
active_temporal_markets_poly.head()

In [ ]:
active_temporal_markets_poly.to_file('active_temporal_markets_poly.geojson', driver='GeoJSON')

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scattermapbox(
        name='Mercados',
        customdata=sjl_old_markets_merged[['NOMBRE_MERCADO', 'aforo']],
        lat=sjl_old_markets_merged.lat,
        lon=sjl_old_markets_merged.lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=sjl_old_markets_merged['aforo_scaled'],
            sizemin=10,
            color='orange',
            opacity=0.5,
        ),
        line={'color': 'black', 'width':50},
        hovertemplate='Nombre:%{customdata[0]} <br><b>Aforo:%{customdata[1]}} ',
        showlegend=True,
    )
)

fig.add_trace(
    go.Choroplethmapbox(
        name='Potenciales mercados itinerantes',
        customdata=active_temporal_markets_poly['address'],
        geojson=active_temporal_markets_poly.geometry.__geo_interface__,
        locations=active_temporal_markets_poly.index, z=active_temporal_markets_poly.active,
        colorscale=[[0, 'rgb(0,255,0)'], [1,'rgb(0,255,0)']],
        showscale=False, 
        showlegend=True,
        marker_opacity=1, marker_line_width=3, marker_line_color='rgb(0,200,0)',
        hovertemplate='Dirección:%{customdata}',
    )
)

fig.update_layout(
    legend={'orientation': 'h'},
    mapbox=dict(
        accesstoken='pk.eyJ1IjoiY2xhdWRpbzk3IiwiYSI6ImNqbzM2NmFtMjB0YnUzd3BvenZzN3QzN3YifQ.heZHwQTY8TWhuO0u2-BxxA',
        center=dict(
            lat=-12.022,
            lon=-76.998,
        ),
        pitch=0,
        bearing=-50,
        zoom=14,
        style='carto-positron'
    ),
)

fig.show()